In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import sys
import csv
import functools
from functools import reduce
import seaborn as sns
import matplotlib.pyplot as plt

csv.field_size_limit(1000000000000000000)

131072

In [2]:
MASDEV = pd.read_pickle("./3_MASTER_DEVICE_File.pkl")

In [3]:
MASDEV.shape
# same as in trial 3

(433946, 30)

In [4]:
# can be used for manufacturer identifier: MANUFACTURER_D_NAME
# can be used for product identifier: BRAND_NAME, GENERIC_NAME, MODEL_NUMBER

In [5]:
manufacCounts = MASDEV['MANUFACTURER_D_NAME'].value_counts(normalize=True).to_frame()
manufacCounts['count'] = MASDEV['MANUFACTURER_D_NAME'].value_counts()
manufacCounts.shape

(743, 2)

In [6]:
manufacCounts[manufacCounts['count'] >= 100].iloc[50:]

,MANUFACTURER_D_NAME,count
SORIN GROUP ITALIA S.R.L.,0.000378,164
OSCOR,0.000355,154
BIOTRONIK SE & CO KG,0.000346,150
ANGEION CORP.,0.000339,147
MEDTRONIC INC,0.000328,142
"SORIN BIOMEDICA C.R.M., S.R.L.",0.000318,138
"CARDIAC PACEMAKERS, INC.",0.000314,136
ST. JUDE MEDICAL INC. CRMD,0.000311,135
CLONMEL,0.000305,132
"ELA MEDICAL, S.A.S.",0.000279,121


In [7]:
# create  STD_MANUFACTURER
MASDEV["STD_MANUFACTURER"] = MASDEV["MANUFACTURER_D_NAME"]
MASDEV.head()

,MDR_REPORT_KEY,REPORT_NUMBER,REPORT_SOURCE_CODE,MANUFACTURER_LINK_FLAG_,ADVERSE_EVENT_FLAG,PRODUCT_PROBLEM_FLAG,DATE_OF_EVENT,REPORTER_OCCUPATION_CODE,HEALTH_PROFESSIONAL,INITIAL_REPORT_TO_FDA,...,GENERIC_NAME,MANUFACTURER_D_NAME,DEVICE_OPERATOR,MODEL_NUMBER,DEVICE_REPORT_PRODUCT_CODE,DEVICE_AGE_TEXT,DEVICE_EVALUATED_BY_MANUFACTUR,DEVICE_SEQUENCE_NO,year,STD_MANUFACTURER
0,257484,2938836-2000-00003,M,Y,Y,N,12/07/1999,001,Y,N,...,IMPLANTABLE CARDIAC DEFIBRILLATOR,"ST. JUDE MEDICAL, INC., CRMD",0HP,V-105C,LWS,3 YR,Y,1,2000,"ST. JUDE MEDICAL, INC., CRMD"
1,257721,MW1017857,P,N,N,Y,12/10/1999,000,Y,NaN,...,IMPLANTABLE CARDIOVERTER DEFIBRILLATOR,ST. JUDE MEDICAL,0HP,V-175-C,LWS,NaN,NaN,1,2000,ST. JUDE MEDICAL
2,257784,257784,U,N,N,Y,12/07/1999,500,Y,Y,...,IMPLANTABLE CARDIAC DEFIBRILLATOR,"MEDTRONIC, INC.",0HP,7271,LWS,*,NaN,1,2000,"MEDTRONIC, INC."
3,257800,2938836-2000-00002,M,Y,Y,N,12/08/1999,001,Y,N,...,IMPLANTABLE CARDIAC DEFIBRILLATOR,"ST. JUDE MEDICAL, INC., CRMD",0HP,V-175C,LWS,9 MO,Y,1,2000,"ST. JUDE MEDICAL, INC., CRMD"
4,257801,2938836-2000-00004,M,Y,Y,N,11/05/1999,001,Y,N,...,IMPLANTABLE CARDIAC DEFIBRILLATOR,"ST. JUDE MEDICAL, INC., CRMD",0HP,V-190HV3,LWS,2 YR,Y,1,2000,"ST. JUDE MEDICAL, INC., CRMD"


In [8]:
# create induvidual manufacturer dfs
# run twice to make sure all have the same length (for merging)

In [13]:
MASDEV["MPRI_MADE"] = pd.DataFrame(MASDEV['MANUFACTURER_D_NAME'].str.contains('MPRI', regex=False))
MASDEV_MPRI = MASDEV[MASDEV["MPRI_MADE"] == True]
MASDEV_MPRI = MASDEV_MPRI.assign(STD_MANUFACTURER='MEDTRONIC')
MASDEV_MPRI.shape

(46085, 35)

In [14]:
MASDEV["MEDTRONIC_MADE"] = pd.DataFrame(MASDEV['MANUFACTURER_D_NAME'].str.contains('MEDTRONIC', regex=False))
MASDEV_MEDTRONIC = MASDEV[MASDEV["MEDTRONIC_MADE"] == True]
MASDEV_MEDTRONIC = MASDEV_MEDTRONIC.assign(STD_MANUFACTURER='MEDTRONIC')
MASDEV_MEDTRONIC.shape

(90290, 35)

In [15]:
MASDEV["BOSTON SCIENTIFIC_MADE"] = pd.DataFrame(MASDEV['MANUFACTURER_D_NAME'].str.contains('BOSTON SCIENTIFIC', regex=False))
MASDEV_BOSTONSCIENTIFIC = MASDEV[MASDEV["BOSTON SCIENTIFIC_MADE"] == True]
MASDEV_BOSTONSCIENTIFIC = MASDEV_BOSTONSCIENTIFIC.assign(STD_MANUFACTURER='BOSTON SCIENTIFIC')
MASDEV_BOSTONSCIENTIFIC.shape

(15960, 35)

In [16]:
MASDEV["ST. JUDE_MADE"] = pd.DataFrame(MASDEV['MANUFACTURER_D_NAME'].str.contains('ST', regex=False) &
                                      MASDEV['MANUFACTURER_D_NAME'].str.contains('JUDE', regex=False))
MASDEV_StJUDE = MASDEV[MASDEV["ST. JUDE_MADE"] == True]
MASDEV_StJUDE = MASDEV_StJUDE.assign(STD_MANUFACTURER='ST. JUDE')
MASDEV_StJUDE.shape

(142433, 35)

In [17]:
MASDEV2 = MASDEV_MPRI
MASDEV2 = MASDEV2.append(MASDEV_MEDTRONIC)
MASDEV2 = MASDEV2.append(MASDEV_StJUDE)
MASDEV2 = MASDEV2.append(MASDEV_BOSTONSCIENTIFIC)
MASDEV2.shape

(294768, 35)

In [18]:
MASDEV2["STD_MANUFACTURER"].value_counts()

ST. JUDE             142433
MEDTRONIC            136375
BOSTON SCIENTIFIC     15960
Name: STD_MANUFACTURER, dtype: int64

In [19]:
# drop duplicate report key values
s=MASDEV2['MDR_REPORT_KEY'].duplicated(keep='first')
indexDupeReportkey = s[s].index
print("there are " + str(len(indexDupeReportkey)) + " duplicate values in MASDEV2")

MASDEV2 = MASDEV2.drop_duplicates(subset=['MDR_REPORT_KEY'], keep='first')

s=MASDEV2['MDR_REPORT_KEY'].duplicated(keep='first')
indexDupeReportkey = s[s].index
print("there are now " + str(len(indexDupeReportkey)) + " duplicate values in MASDEV2 after removal")
print("the shape of MASDEV2 is now: " + str(MASDEV2.shape))

there are 2 duplicate values in MASDEV2
there are now 0 duplicate values in MASDEV2 after removal
the shape of MASDEV2 is now: (294766, 35)


In [20]:
# remove unwanted columns
print("the shape of MASDEV2 is: " + str(MASDEV2.shape))

keptCols = ['MDR_REPORT_KEY',
            'REPORT_NUMBER',
            'REPORT_SOURCE_CODE',
            'MANUFACTURER_LINK_FLAG_',
            'ADVERSE_EVENT_FLAG',
            'PRODUCT_PROBLEM_FLAG',
            'DATE_OF_EVENT',
            'REPORTER_OCCUPATION_CODE',
            'HEALTH_PROFESSIONAL',
            'INITIAL_REPORT_TO_FDA',
            'REPORT_TO_FDA',
            'EVENT_LOCATION',
            'EVENT_TYPE',
            'TYPE_OF_REPORT',
            'DATE_ADDED',
            'DATE_CHANGED',
            'REPORTER_COUNTRY_CODE',
            'SUMMARY_REPORT',
            'DEVICE_SEQUENCE_NO',
            'DATE_RECEIVED',
            'BRAND_NAME',
            'GENERIC_NAME',
            'DEVICE_OPERATOR',
            'MODEL_NUMBER',
            'DEVICE_REPORT_PRODUCT_CODE',
            'DEVICE_AGE_TEXT',
            'DEVICE_EVALUATED_BY_MANUFACTUR',
            'year',
            'STD_MANUFACTURER']

MASDEV2 = MASDEV2[keptCols]
print("the shape of MASDEV2 after column removal is: " + str(MASDEV2.shape))
print(MASDEV2.columns)

the shape of MASDEV2 is: (294766, 35)
the shape of MASDEV2 after column removal is: (294766, 30)
Index(['MDR_REPORT_KEY', 'REPORT_NUMBER', 'REPORT_SOURCE_CODE',
       'MANUFACTURER_LINK_FLAG_', 'ADVERSE_EVENT_FLAG', 'PRODUCT_PROBLEM_FLAG',
       'DATE_OF_EVENT', 'REPORTER_OCCUPATION_CODE', 'HEALTH_PROFESSIONAL',
       'INITIAL_REPORT_TO_FDA', 'REPORT_TO_FDA', 'EVENT_LOCATION',
       'EVENT_TYPE', 'TYPE_OF_REPORT', 'DATE_ADDED', 'DATE_CHANGED',
       'REPORTER_COUNTRY_CODE', 'SUMMARY_REPORT', 'DEVICE_SEQUENCE_NO',
       'DEVICE_SEQUENCE_NO', 'DATE_RECEIVED', 'BRAND_NAME', 'GENERIC_NAME',
       'DEVICE_OPERATOR', 'MODEL_NUMBER', 'DEVICE_REPORT_PRODUCT_CODE',
       'DEVICE_AGE_TEXT', 'DEVICE_EVALUATED_BY_MANUFACTUR', 'year',
       'STD_MANUFACTURER'],
      dtype='object')


In [21]:
# create the standardized device identifier and recall class column
MASDEV2["STD_DEVICE"] = MASDEV2['BRAND_NAME']
MASDEV2["RECALL_CLASS"] = MASDEV2['BRAND_NAME']
MASDEV2 = MASDEV2.assign(RECALL_CLASS="0")
MASDEV2.head()

,MDR_REPORT_KEY,REPORT_NUMBER,REPORT_SOURCE_CODE,MANUFACTURER_LINK_FLAG_,ADVERSE_EVENT_FLAG,PRODUCT_PROBLEM_FLAG,DATE_OF_EVENT,REPORTER_OCCUPATION_CODE,HEALTH_PROFESSIONAL,INITIAL_REPORT_TO_FDA,...,GENERIC_NAME,DEVICE_OPERATOR,MODEL_NUMBER,DEVICE_REPORT_PRODUCT_CODE,DEVICE_AGE_TEXT,DEVICE_EVALUATED_BY_MANUFACTUR,year,STD_MANUFACTURER,STD_DEVICE,RECALL_CLASS
7089,489687,2649622-2003-00267,M,Y,N,*,03/06/2003,*,*,N,...,*,*,6944,LWS,NaN,Y,2003,MEDTRONIC,*,0
7514,507667,2649622-2003-00355,M,Y,N,*,03/13/2003,*,*,*,...,*,*,*,LWS,NaN,*,2003,MEDTRONIC,*,0
17701,687828,2649622-2006-00441,M,Y,Y,N,11/18/2005,001,Y,N,...,"DEFIBRILLATOR, AUTOMATIC IMPLANTABLE CARDIOVERTER",0LP,6949,LWS,8 MO,Y,2006,MEDTRONIC,SPRINT FIDELIS,0
19469,734998,2649622-2006-00783,M,Y,Y,N,05/23/2006,600,N,N,...,"DEFIBRILLATOR, AUTOMATIC IMPLANTABLE CARDIOVERTER",0LP,6931,LWS,12 MO,Y,2006,MEDTRONIC,SPRINT FIDELIS,0
27519,849636,2649622-2007-01218,M,Y,Y,Y,03/13/2007,305,Y,N,...,"DEFIBRILLATOR, AUTOMATIC IMPLANTABLE CARDIOVERTER",0LP,6949,LWS,24 MO,*,2007,MEDTRONIC,SPRINT FIDELIS,0


In [22]:
# creating product dfs
# run twice to merge

In [51]:
MASDEV2["MEDTRONIC_GEM_DR"] = pd.DataFrame(MASDEV2['BRAND_NAME'].str.contains('GEM', regex=False) &
                                        MASDEV2['BRAND_NAME'].str.contains('DR', regex=False) &
                                        MASDEV2['STD_MANUFACTURER'].str.contains('MEDTRONIC', regex=False))
MASDEV2_MEDTRONIC_GEM_DR = MASDEV2[MASDEV2["MEDTRONIC_GEM_DR"] == True]
MASDEV2_MEDTRONIC_GEM_DR = MASDEV2_MEDTRONIC_GEM_DR.assign(STD_DEVICE='MEDTRONIC_GEM_DR', RECALL_CLASS="1")
MASDEV2_MEDTRONIC_GEM_DR.shape

(817, 59)

In [52]:
MASDEV2["MEDTRONIC_COMPIA_MRI"] = pd.DataFrame(MASDEV2['BRAND_NAME'].str.contains('COMPIA', regex=False) &
                                        MASDEV2['BRAND_NAME'].str.contains('MRI', regex=False) &
                                        MASDEV2['STD_MANUFACTURER'].str.contains('MEDTRONIC', regex=False))
MASDEV2_MEDTRONIC_COMPIA_MRI = MASDEV2[MASDEV2["MEDTRONIC_COMPIA_MRI"] == True]
MASDEV2_MEDTRONIC_COMPIA_MRI = MASDEV2_MEDTRONIC_COMPIA_MRI.assign(STD_DEVICE='MEDTRONIC_COMPIA_MRI', RECALL_CLASS="1")
MASDEV2_MEDTRONIC_COMPIA_MRI.shape

(262, 59)

In [53]:
MASDEV2["MEDTRONIC_AMPLIA_MRI"] = pd.DataFrame(MASDEV2['BRAND_NAME'].str.contains('AMPLIA', regex=False) &
                                        MASDEV2['BRAND_NAME'].str.contains('MRI', regex=False) &
                                        MASDEV2['STD_MANUFACTURER'].str.contains('MEDTRONIC', regex=False))
MASDEV2_MEDTRONIC_AMPLIA_MRI = MASDEV2[MASDEV2["MEDTRONIC_AMPLIA_MRI"] == True]
MASDEV2_MEDTRONIC_AMPLIA_MRI = MASDEV2_MEDTRONIC_AMPLIA_MRI.assign(STD_DEVICE='MEDTRONIC_AMPLIA_MRI', RECALL_CLASS="1")
MASDEV2_MEDTRONIC_AMPLIA_MRI.shape

(1355, 59)

In [54]:
MASDEV2["MEDTRONIC_VIVA"] = pd.DataFrame(MASDEV2['BRAND_NAME'].str.contains('VIVA', regex=False) &
                                        MASDEV2['STD_MANUFACTURER'].str.contains('MEDTRONIC', regex=False))
MASDEV2_MEDTRONIC_VIVA = MASDEV2[MASDEV2["MEDTRONIC_VIVA"] == True]
MASDEV2_MEDTRONIC_VIVA = MASDEV2_MEDTRONIC_VIVA.assign(STD_DEVICE='MEDTRONIC_VIVA', RECALL_CLASS="1")
MASDEV2_MEDTRONIC_VIVA.shape

(4964, 59)

In [55]:
MASDEV2["MEDTRONIC_BRAVA"] = pd.DataFrame(MASDEV2['BRAND_NAME'].str.contains('BRAVA', regex=False) &
                                        MASDEV2['STD_MANUFACTURER'].str.contains('MEDTRONIC', regex=False))
MASDEV2_MEDTRONIC_BRAVA = MASDEV2[MASDEV2["MEDTRONIC_BRAVA"] == True]
MASDEV2_MEDTRONIC_BRAVA = MASDEV2_MEDTRONIC_BRAVA.assign(STD_DEVICE='MEDTRONIC_BRAVA', RECALL_CLASS="1")
MASDEV2_MEDTRONIC_BRAVA.shape

(208, 59)

In [56]:
MASDEV2["MEDTRONIC_EGIDA"] = pd.DataFrame(MASDEV2['BRAND_NAME'].str.contains('EGIDA', regex=False) &
                                        MASDEV2['STD_MANUFACTURER'].str.contains('MEDTRONIC', regex=False))
MASDEV2_MEDTRONIC_EGIDA = MASDEV2[MASDEV2["MEDTRONIC_EGIDA"] == True]
MASDEV2_MEDTRONIC_EGIDA = MASDEV2_MEDTRONIC_EGIDA.assign(STD_DEVICE='MEDTRONIC_EGIDA', RECALL_CLASS="0")
MASDEV2_MEDTRONIC_EGIDA.shape

(130, 59)

In [58]:
MASDEV2["StJUDE_QUARTET"] = pd.DataFrame(MASDEV2['BRAND_NAME'].str.contains('QUARTET', regex=False) &
                                        MASDEV2['STD_MANUFACTURER'].str.contains('ST. JUDE', regex=False))

MASDEV2_StJUDE_QUARTET = MASDEV2[MASDEV2["StJUDE_QUARTET"] == True]
MASDEV2_StJUDE_QUARTET = MASDEV2_StJUDE_QUARTET.assign(STD_DEVICE='StJUDE_QUARTET', RECALL_CLASS="0")
MASDEV2_StJUDE_QUARTET.shape

(2029, 59)

In [59]:
MASDEV2["StJUDE_GALLANT"] = pd.DataFrame(MASDEV2['BRAND_NAME'].str.contains('GALLANT', regex=False) &
                                        MASDEV2['STD_MANUFACTURER'].str.contains('ST. JUDE', regex=False))

MASDEV2_StJUDE_GALLANT = MASDEV2[MASDEV2["StJUDE_GALLANT"] == True]
MASDEV2_StJUDE_GALLANT = MASDEV2_StJUDE_GALLANT.assign(STD_DEVICE='StJUDE_GALLANT', RECALL_CLASS="0")
MASDEV2_StJUDE_GALLANT.shape

(26, 59)

In [60]:
MASDEV2["StJUDE_DURATA"] = pd.DataFrame(MASDEV2['BRAND_NAME'].str.contains('DURATA', regex=False) &
                                        MASDEV2['STD_MANUFACTURER'].str.contains('ST. JUDE', regex=False))

MASDEV2_StJUDE_DURATA = MASDEV2[MASDEV2["StJUDE_DURATA"] == True]
MASDEV2_StJUDE_DURATA = MASDEV2_StJUDE_DURATA.assign(STD_DEVICE='StJUDE_DURATA', RECALL_CLASS="0")
MASDEV2_StJUDE_DURATA.shape

(26277, 59)

In [61]:
MASDEV2["BOS_SCI_EMBLEM"] = pd.DataFrame(MASDEV2['BRAND_NAME'].str.contains('EMBLEM', regex=False) &
                                        MASDEV2['STD_MANUFACTURER'].str.contains('BOSTON', regex=False))

MASDEV2_BOS_SCI_EMBLEM = MASDEV2[MASDEV2["BOS_SCI_EMBLEM"] == True]
MASDEV2_BOS_SCI_EMBLEM = MASDEV2_BOS_SCI_EMBLEM.assign(STD_DEVICE='BOS_SCI_EMBLEM', RECALL_CLASS="1")
MASDEV2_BOS_SCI_EMBLEM.shape

(3522, 59)

In [62]:
MASDEV2["BOS_SCI_INOGEN_EL"] = pd.DataFrame(MASDEV2['BRAND_NAME'].str.contains('INOGEN', regex=False) &
                                         MASDEV2["BRAND_NAME"].str.contains("EL", regex=False) &
                                        MASDEV2['STD_MANUFACTURER'].str.contains('BOSTON', regex=False))

MASDEV2_BOS_SCI_INOGEN_EL = MASDEV2[MASDEV2["BOS_SCI_INOGEN_EL"] == True]
MASDEV2_BOS_SCI_INOGEN_EL = MASDEV2_BOS_SCI_INOGEN_EL.assign(STD_DEVICE='BOS_SCI_INOGEN_EL', RECALL_CLASS="0")
MASDEV2_BOS_SCI_INOGEN_EL.shape

(533, 59)

In [63]:
MASDEV2["BOS_SCI_PERCIVA"] = pd.DataFrame(MASDEV2['BRAND_NAME'].str.contains('PERCIVA', regex=False) &
                                        MASDEV2['STD_MANUFACTURER'].str.contains('BOSTON', regex=False))

MASDEV2_BOS_SCI_PERCIVA = MASDEV2[MASDEV2["BOS_SCI_PERCIVA"] == True]
MASDEV2_BOS_SCI_PERCIVA = MASDEV2_BOS_SCI_PERCIVA.assign(STD_DEVICE='BOS_SCI_PERCIVA', RECALL_CLASS="0")
MASDEV2_BOS_SCI_PERCIVA.shape

(39, 59)

In [64]:
MASDEV2["MEDTRONIC_EVERA"] = pd.DataFrame(MASDEV2['BRAND_NAME'].str.contains('EVERA', regex=False)&
                                        MASDEV2['STD_MANUFACTURER'].str.contains('MEDTRONIC', regex=False))

MASDEV2_MEDTRONIC_EVERA = MASDEV2[MASDEV2["MEDTRONIC_EVERA"] == True]
MASDEV2_MEDTRONIC_EVERA = MASDEV2_MEDTRONIC_EVERA.assign(STD_DEVICE='MEDTRONIC_EVERA', RECALL_CLASS="1")
MASDEV2_MEDTRONIC_EVERA.shape

(6292, 59)

In [65]:
MASDEV2["MEDTRONIC_MICRO_JEWEL"] = pd.DataFrame(MASDEV2['BRAND_NAME'].str.contains('MICRO', regex=False) &
                                        MASDEV2["BRAND_NAME"].str.contains("JEWEL", regex=False) &
                                        MASDEV2['STD_MANUFACTURER'].str.contains('MEDTRONIC', regex=False))
MASDEV2_MEDTRONIC_MICRO_JEWEL = MASDEV2[MASDEV2["MEDTRONIC_MICRO_JEWEL"] == True]
MASDEV2_MEDTRONIC_MICRO_JEWEL = MASDEV2_MEDTRONIC_MICRO_JEWEL.assign(STD_DEVICE='MEDTRONIC_MICRO_JEWEL', RECALL_CLASS="1")
MASDEV2_MEDTRONIC_MICRO_JEWEL.shape

(609, 59)

In [66]:
MASDEV2["MEDTRONIC_PROTECTA_XT"] = pd.DataFrame(MASDEV2['BRAND_NAME'].str.contains('PROTECTA', regex=False) &
                                        MASDEV2["BRAND_NAME"].str.contains("XT", regex=False) &
                                        MASDEV2['STD_MANUFACTURER'].str.contains('MEDTRONIC', regex=False))
MASDEV2_MEDTRONIC_PROTECTA_XT = MASDEV2[MASDEV2["MEDTRONIC_PROTECTA_XT"] == True]
MASDEV2_MEDTRONIC_PROTECTA_XT = MASDEV2_MEDTRONIC_PROTECTA_XT.assign(STD_DEVICE='MEDTRONIC_PROTECTA_XT', RECALL_CLASS="0")
MASDEV2_MEDTRONIC_PROTECTA_XT.shape

(4891, 59)

In [67]:
MASDEV2["MEDTRONIC_SPRINT"] = pd.DataFrame(MASDEV2['BRAND_NAME'].str.contains('SPRINT', regex=False) &
                                          (MASDEV2["BRAND_NAME"].str.contains("FID", regex=False) == False) &
                                          (MASDEV2["BRAND_NAME"].str.contains("FIELIS", regex=False) == False) &
                                          (MASDEV2["BRAND_NAME"].str.contains("FEDELIS", regex=False) == False) &
                                          ((MASDEV2["BRAND_NAME"].str.contains(" QUAT", regex=False) == False) &
                                           (MASDEV2["BRAND_NAME"].str.contains("QUADRO", regex=False) == False)) &
                                        MASDEV2['STD_MANUFACTURER'].str.contains('MEDTRONIC', regex=False))
MASDEV2_MEDTRONIC_SPRINT = MASDEV2[MASDEV2["MEDTRONIC_SPRINT"] == True]
MASDEV2_MEDTRONIC_SPRINT = MASDEV2_MEDTRONIC_SPRINT.assign(STD_DEVICE='MEDTRONIC_SPRINT', RECALL_CLASS="0")
MASDEV2_MEDTRONIC_SPRINT.shape

(4593, 59)

In [68]:
MASDEV2["MEDTRONIC_SPRINT_FIDELIS"] = pd.DataFrame(MASDEV2['BRAND_NAME'].str.contains('SPRINT', regex=False) &
                                                    MASDEV2["BRAND_NAME"].str.contains("FID", regex=False) &
                                                    MASDEV2['STD_MANUFACTURER'].str.contains('MEDTRONIC', regex=False))
MASDEV2_MEDTRONIC_SPRINT_FIDELIS = MASDEV2[MASDEV2["MEDTRONIC_SPRINT_FIDELIS"] == True]
MASDEV2_MEDTRONIC_SPRINT_FIDELIS = MASDEV2_MEDTRONIC_SPRINT_FIDELIS.assign(STD_DEVICE='MEDTRONIC_SPRINT_FIDELIS')
MASDEV2_MEDTRONIC_SPRINT_FIDELIS = MASDEV2_MEDTRONIC_SPRINT_FIDELIS.assign(RECALL_CLASS='1')
MASDEV2_MEDTRONIC_SPRINT_FIDELIS.shape

(14329, 59)

In [69]:
MASDEV2["MEDTRONIC_SPRINT_QUATTRO"] = pd.DataFrame(MASDEV2['BRAND_NAME'].str.contains('SPRINT', regex=False) &
                                                            MASDEV2["BRAND_NAME"].str.contains("QU", regex=False) &
                                                            (MASDEV2["BRAND_NAME"].str.contains("SECURE", regex=False) == False) &
                                                            MASDEV2['STD_MANUFACTURER'].str.contains('MEDTRONIC', regex=False))
MASDEV2_MEDTRONIC_SPRINT_QUATTRO = MASDEV2[MASDEV2["MEDTRONIC_SPRINT_QUATTRO"] == True]
MASDEV2_MEDTRONIC_SPRINT_QUATTRO = MASDEV2_MEDTRONIC_SPRINT_QUATTRO.assign(STD_DEVICE='MEDTRONIC_SPRINT_QUATTRO', RECALL_CLASS="0")
MASDEV2_MEDTRONIC_SPRINT_QUATTRO.shape

(6028, 59)

In [70]:
MASDEV2["MEDTRONIC_SPRINT_QUATTRO_SECURE_MRI"] = pd.DataFrame(MASDEV2['BRAND_NAME'].str.contains('SPRINT', regex=False) &
                                                            MASDEV2["BRAND_NAME"].str.contains("QU", regex=False) &
                                                            MASDEV2["BRAND_NAME"].str.contains("SECURE", regex=False) &
                                                            MASDEV2["BRAND_NAME"].str.contains(" MRI", regex=False) &
                                                            MASDEV2['STD_MANUFACTURER'].str.contains('MEDTRONIC', regex=False))
MASDEV2_MEDTRONIC_SPRINT_QUATTRO_SECURE_MRI = MASDEV2[MASDEV2["MEDTRONIC_SPRINT_QUATTRO_SECURE_MRI"] == True]
MASDEV2_MEDTRONIC_SPRINT_QUATTRO_SECURE_MRI = MASDEV2_MEDTRONIC_SPRINT_QUATTRO_SECURE_MRI.assign(STD_DEVICE='MEDTRONIC_SPRINT_QUATTRO_SECURE_MRI', RECALL_CLASS="0")
MASDEV2_MEDTRONIC_SPRINT_QUATTRO_SECURE_MRI.shape

(10460, 59)

In [71]:
MASDEV2["MEDTRONIC_TRANSVENE"] = pd.DataFrame(MASDEV2['BRAND_NAME'].str.contains('TRANSVENE', regex=False) &
                                        MASDEV2['STD_MANUFACTURER'].str.contains('MEDTRONIC', regex=False))
MASDEV2_MEDTRONIC_TRANSVENE = MASDEV2[MASDEV2["MEDTRONIC_TRANSVENE"] == True]
MASDEV2_MEDTRONIC_TRANSVENE = MASDEV2_MEDTRONIC_TRANSVENE.assign(STD_DEVICE='MEDTRONIC_TRANSVENE', RECALL_CLASS="0")
MASDEV2_MEDTRONIC_TRANSVENE.shape

(2200, 59)

In [72]:
MASDEV2["MEDTRONIC_SUBQ"] = pd.DataFrame(MASDEV2['BRAND_NAME'].str.contains('SUB', regex=False) &
                                        MASDEV2["BRAND_NAME"].str.contains("Q", regex=False) &
                                        MASDEV2['STD_MANUFACTURER'].str.contains('MEDTRONIC', regex=False))
MASDEV2_MEDTRONIC_SUBQ = MASDEV2[MASDEV2["MEDTRONIC_SUBQ"] == True]
MASDEV2_MEDTRONIC_SUBQ = MASDEV2_MEDTRONIC_SUBQ.assign(STD_DEVICE='MEDTRONIC_SUBQ', RECALL_CLASS="0")
MASDEV2_MEDTRONIC_SUBQ.shape

(293, 59)

In [73]:
MASDEV2["MEDTRONIC_VISIA"] = pd.DataFrame(MASDEV2['BRAND_NAME'].str.contains('VISIA', regex=False)&
                                        MASDEV2['STD_MANUFACTURER'].str.contains('MEDTRONIC', regex=False))

MASDEV2_MEDTRONIC_VISIA = MASDEV2[MASDEV2["MEDTRONIC_VISIA"] == True]
MASDEV2_MEDTRONIC_VISIA = MASDEV2_MEDTRONIC_VISIA.assign(STD_DEVICE='MEDTRONIC_VISIA', RECALL_CLASS="1")
MASDEV2_MEDTRONIC_VISIA.shape

(1057, 59)

In [74]:
MASDEV2["StJUDE_ELLIPSE"] = pd.DataFrame(MASDEV2['BRAND_NAME'].str.contains('ELLIPSE', regex=False) &
                                        MASDEV2['STD_MANUFACTURER'].str.contains('ST. JUDE', regex=False))

MASDEV2_StJUDE_ELLIPSE = MASDEV2[MASDEV2["StJUDE_ELLIPSE"] == True]
MASDEV2_StJUDE_ELLIPSE = MASDEV2_StJUDE_ELLIPSE.assign(STD_DEVICE='StJUDE_ELLIPSE', RECALL_CLASS="1")
MASDEV2_StJUDE_ELLIPSE.shape

(5074, 59)

In [75]:
MASDEV2["STJUDE_FORTIFY_VR"] = pd.DataFrame(MASDEV2['BRAND_NAME'].str.contains('FORTIFY', regex=False) &
                                          MASDEV2["BRAND_NAME"].str.contains("VR", regex=False) &
                                        MASDEV2['STD_MANUFACTURER'].str.contains('ST. JUDE', regex=False))
MASDEV2_STJUDE_FORTIFY_VR = MASDEV2[MASDEV2["STJUDE_FORTIFY_VR"] == True]
MASDEV2_STJUDE_FORTIFY_VR = MASDEV2_STJUDE_FORTIFY_VR.assign(STD_DEVICE='STJUDE_FORTIFY_VR')
MASDEV2_STJUDE_FORTIFY_VR = MASDEV2_STJUDE_FORTIFY_VR.assign(RECALL_CLASS="1")
MASDEV2_STJUDE_FORTIFY_VR.shape

(6597, 59)

In [76]:
MASDEV2["STJUDE_RIATA"] = pd.DataFrame(MASDEV2['BRAND_NAME'].str.contains('RIATA', regex=False) &
                                      (MASDEV2["BRAND_NAME"].str.contains("ST", regex=False) == False) &
                                        MASDEV2['STD_MANUFACTURER'].str.contains('ST. JUDE', regex=False))
MASDEV2_STJUDE_RIATA = MASDEV2[MASDEV2["STJUDE_RIATA"] == True]
MASDEV2_STJUDE_RIATA = MASDEV2_STJUDE_RIATA.assign(STD_DEVICE='STJUDE_RIATA')
MASDEV2_STJUDE_RIATA = MASDEV2_STJUDE_RIATA.assign(RECALL_CLASS="1")
MASDEV2_STJUDE_RIATA.shape

(17992, 59)

In [77]:
MASDEV2["STJUDE_RIATA_ST"] = pd.DataFrame(MASDEV2['BRAND_NAME'].str.contains('RIATA', regex=False) &
                                            MASDEV2["BRAND_NAME"].str.contains("ST", regex=False) &
                                            (MASDEV2["BRAND_NAME"].str.contains("OPTIM", regex=False) == False) &
                                            MASDEV2['STD_MANUFACTURER'].str.contains('ST. JUDE', regex=False))
MASDEV2_STJUDE_RIATA_ST = MASDEV2[MASDEV2["STJUDE_RIATA_ST"] == True]
MASDEV2_STJUDE_RIATA_ST = MASDEV2_STJUDE_RIATA_ST.assign(STD_DEVICE='STJUDE_RIATA_ST')
MASDEV2_STJUDE_RIATA_ST = MASDEV2_STJUDE_RIATA_ST.assign(RECALL_CLASS="1")
MASDEV2_STJUDE_RIATA_ST.shape

(6321, 59)

In [78]:
MASDEV2["STJUDE_TENDRIL_STS"] = pd.DataFrame(MASDEV2['BRAND_NAME'].str.contains('TENDRIL', regex=False) &
                                      (MASDEV2["BRAND_NAME"].str.contains("STS", regex=False) == False) &
                                        MASDEV2['STD_MANUFACTURER'].str.contains('ST. JUDE', regex=False))
MASDEV2_STJUDE_TENDRIL_STS = MASDEV2[MASDEV2["STJUDE_TENDRIL_STS"] == True]
MASDEV2_STJUDE_TENDRIL_STS = MASDEV2_STJUDE_TENDRIL_STS.assign(STD_DEVICE='STJUDE_TENDRIL_STS')
MASDEV2_STJUDE_TENDRIL_STS = MASDEV2_STJUDE_TENDRIL_STS.assign(RECALL_CLASS="0")
MASDEV2_STJUDE_TENDRIL_STS.shape

(7, 59)

In [79]:
# merge product dfs
MASDEV3 = MASDEV2_BOS_SCI_EMBLEM

for df in [MASDEV2_BOS_SCI_INOGEN_EL, MASDEV2_BOS_SCI_PERCIVA, MASDEV2_MEDTRONIC_EVERA, MASDEV2_MEDTRONIC_MICRO_JEWEL,
            MASDEV2_MEDTRONIC_PROTECTA_XT, MASDEV2_MEDTRONIC_SPRINT, MASDEV2_MEDTRONIC_SPRINT_FIDELIS, MASDEV2_MEDTRONIC_SPRINT_QUATTRO,
            MASDEV2_MEDTRONIC_SPRINT_QUATTRO_SECURE_MRI, MASDEV2_MEDTRONIC_TRANSVENE, MASDEV2_MEDTRONIC_SUBQ, 
            MASDEV2_MEDTRONIC_VISIA, MASDEV2_StJUDE_ELLIPSE, MASDEV2_STJUDE_FORTIFY_VR, MASDEV2_STJUDE_RIATA,
            MASDEV2_STJUDE_RIATA_ST, MASDEV2_STJUDE_TENDRIL_STS, MASDEV2_StJUDE_DURATA, MASDEV2_StJUDE_GALLANT,
            MASDEV2_StJUDE_QUARTET, MASDEV2_MEDTRONIC_PROTECTA_XT, MASDEV2_MEDTRONIC_EGIDA,
            MASDEV2_MEDTRONIC_BRAVA, MASDEV2_MEDTRONIC_VIVA, MASDEV2_MEDTRONIC_AMPLIA_MRI, 
            MASDEV2_MEDTRONIC_COMPIA_MRI, MASDEV2_MEDTRONIC_GEM_DR]:
    MASDEV3 = MASDEV3.append(df)
    print(MASDEV3.shape)

(4055, 59)
(4094, 59)
(10386, 59)
(10995, 59)
(15886, 59)
(20479, 59)
(34808, 59)
(40836, 59)
(51296, 59)
(53496, 59)
(53789, 59)
(54846, 59)
(59920, 59)
(66517, 59)
(84509, 59)
(90830, 59)
(90837, 59)
(117114, 59)
(117140, 59)
(119169, 59)
(124060, 59)
(124190, 59)
(124398, 59)
(129362, 59)
(130717, 59)
(130979, 59)
(131796, 59)


In [83]:
# check value counts
MASDEV3["STD_DEVICE"].value_counts()

StJUDE_DURATA                          26277
STJUDE_RIATA                           17992
MEDTRONIC_SPRINT_FIDELIS               14329
MEDTRONIC_SPRINT_QUATTRO_SECURE_MRI    10460
MEDTRONIC_PROTECTA_XT                   9782
STJUDE_FORTIFY_VR                       6597
STJUDE_RIATA_ST                         6321
MEDTRONIC_EVERA                         6292
MEDTRONIC_SPRINT_QUATTRO                6028
StJUDE_ELLIPSE                          5074
MEDTRONIC_VIVA                          4964
MEDTRONIC_SPRINT                        4593
BOS_SCI_EMBLEM                          3522
MEDTRONIC_TRANSVENE                     2200
StJUDE_QUARTET                          2029
MEDTRONIC_AMPLIA_MRI                    1355
MEDTRONIC_VISIA                         1057
MEDTRONIC_GEM_DR                         817
MEDTRONIC_MICRO_JEWEL                    609
BOS_SCI_INOGEN_EL                        533
MEDTRONIC_SUBQ                           293
MEDTRONIC_COMPIA_MRI                     262
MEDTRONIC_

In [84]:
MASDEV3["RECALL_CLASS"].value_counts()

1    69399
0    62397
Name: RECALL_CLASS, dtype: int64

In [85]:
# remove unwanted columns
print("the shape of MASDEV3 is: " + str(MASDEV3.shape))

keptCols = ['MDR_REPORT_KEY',
            'REPORT_NUMBER',
            'REPORT_SOURCE_CODE',
            'MANUFACTURER_LINK_FLAG_',
            'ADVERSE_EVENT_FLAG',
            'PRODUCT_PROBLEM_FLAG',
            'DATE_OF_EVENT',
            'REPORTER_OCCUPATION_CODE',
            'HEALTH_PROFESSIONAL',
            'INITIAL_REPORT_TO_FDA',
            'REPORT_TO_FDA',
            'EVENT_LOCATION',
            'EVENT_TYPE',
            'TYPE_OF_REPORT',
            'DATE_ADDED',
            'DATE_CHANGED',
            'REPORTER_COUNTRY_CODE',
            'SUMMARY_REPORT',
            'DEVICE_SEQUENCE_NO',
            'DATE_RECEIVED',
            'BRAND_NAME',
            'GENERIC_NAME',
            'DEVICE_OPERATOR',
            'MODEL_NUMBER',
            'DEVICE_REPORT_PRODUCT_CODE',
            'DEVICE_AGE_TEXT',
            'DEVICE_EVALUATED_BY_MANUFACTUR',
            'year',
            'STD_MANUFACTURER',
            'STD_DEVICE',
            'RECALL_CLASS']

MASDEV3 = MASDEV3[keptCols]
print("the shape of MASDEV3 after column removal is: " + str(MASDEV3.shape))
print(MASDEV3.columns)

the shape of MASDEV3 is: (131796, 59)
the shape of MASDEV3 after column removal is: (131796, 32)
Index(['MDR_REPORT_KEY', 'REPORT_NUMBER', 'REPORT_SOURCE_CODE',
       'MANUFACTURER_LINK_FLAG_', 'ADVERSE_EVENT_FLAG', 'PRODUCT_PROBLEM_FLAG',
       'DATE_OF_EVENT', 'REPORTER_OCCUPATION_CODE', 'HEALTH_PROFESSIONAL',
       'INITIAL_REPORT_TO_FDA', 'REPORT_TO_FDA', 'EVENT_LOCATION',
       'EVENT_TYPE', 'TYPE_OF_REPORT', 'DATE_ADDED', 'DATE_CHANGED',
       'REPORTER_COUNTRY_CODE', 'SUMMARY_REPORT', 'DEVICE_SEQUENCE_NO',
       'DEVICE_SEQUENCE_NO', 'DATE_RECEIVED', 'BRAND_NAME', 'GENERIC_NAME',
       'DEVICE_OPERATOR', 'MODEL_NUMBER', 'DEVICE_REPORT_PRODUCT_CODE',
       'DEVICE_AGE_TEXT', 'DEVICE_EVALUATED_BY_MANUFACTUR', 'year',
       'STD_MANUFACTURER', 'STD_DEVICE', 'RECALL_CLASS'],
      dtype='object')


In [86]:
MASDEV3.to_pickle("./4_MASDEV_WithStdLabels.pkl")